1

In [ ]:
# region_rf_shape_scaled.py - Region-wise Shape Forecasting using Random Forest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

# -------------------------------------------------------------
# 1. File Paths
# -------------------------------------------------------------
TRAIN_XLSX = "/content/Hourly_Load_Data_2018_2022_LeapFixed.xlsx"
TEST_XLSX = "/content/India_2023_Hourly_Load_Data.xlsx"
ANNUAL_XLSX = "/content/Regional_Peak___Energy__2018_2023_ &2030.xlsx"

# -------------------------------------------------------------
# 2. Load Data
# -------------------------------------------------------------
def load_data(file, years, sheet_name=None):
    xl = pd.ExcelFile(file)
    dfs = []
    for y in years:
        try:
            df = xl.parse(str(y))
        except:
            df = xl.parse(sheet_name or "Sheet1")
        df["Year"] = y
        df["ts"] = pd.date_range(f"{y}-01-01 00:00", periods=len(df), freq="h")
        dfs.append(df.set_index("ts"))
    return pd.concat(dfs)

train_raw = load_data(TRAIN_XLSX, list(range(2018, 2023)))
test_raw = load_data(TEST_XLSX, [2023], sheet_name="Sheet1")

region_cols = ["Northern Region", "North Eastern Region", "Eastern Region",
               "Southern Region", "Western Region"]

# -------------------------------------------------------------
# 3. Load Annual Data
# -------------------------------------------------------------
ann = pd.read_excel(ANNUAL_XLSX, engine="openpyxl")
ann.columns = [c.strip().lower() for c in ann.columns]
for c in ann.columns:
    if "peak" in c: ann = ann.rename(columns={c: "year_peak"})
    if "energy" in c: ann = ann.rename(columns={c: "year_energy"})
ann = ann.rename(columns={"year": "Year"})
ann["Year"] = ann["Year"].astype(int)
ann = ann.drop_duplicates(subset="Year").set_index("Year")

for df in [train_raw, test_raw]:
    df["year_peak"] = df["Year"].map(ann["year_peak"])
    df["year_energy"] = df["Year"].map(ann["year_energy"])

# -------------------------------------------------------------
# 4. Feature Engineering (shape-based, no lags)
# -------------------------------------------------------------
def make_features(df):
    feats = pd.DataFrame(index=df.index)
    feats["hour"] = df.index.hour
    feats["dow"] = df.index.dayofweek
    feats["month"] = df.index.month
    feats["wknd"] = (feats["dow"] >= 5).astype(int)
    feats["year_peak"] = df["year_peak"]
    feats["year_energy"] = df["year_energy"]
    feats["target"] = df["Load"] / df["year_peak"]  # Normalized shape
    return feats.dropna()

# -------------------------------------------------------------
# 5. Train + Evaluate Shape Model with Scaling
# -------------------------------------------------------------
all_metrics = {}
all_preds = {}

for reg in region_cols:
    print(f"\n=== Training Shape Model (RF): {reg} ===")
    train = train_raw.copy()
    test = test_raw.copy()

    train["Load"] = train[reg]
    test["Load"] = test[reg]

    feat_train = make_features(train)
    feat_test = make_features(test)

    X_tr, y_tr = feat_train.drop("target", axis=1), feat_train["target"]
    X_te, y_te = feat_test.drop("target", axis=1), feat_test["target"]

    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=12,
        min_samples_split=10,
        max_features="sqrt",
        n_jobs=-1,
        random_state=42
    )
    model.fit(X_tr, y_tr)
    shape_hat = model.predict(X_te)

    # Rescale using 2023 peak
    peak2023 = test["year_peak"].iloc[0]
    y_hat = shape_hat * peak2023
    y_true = test["Load"].loc[X_te.index]  # aligned actuals

    rmse = np.sqrt(mean_squared_error(y_true, y_hat))
    mape = mean_absolute_percentage_error(y_true, y_hat) * 100
    mbe = 100 * (y_hat - y_true).mean() / y_true.mean()
    r2 = r2_score(y_true, y_hat)

    all_metrics[reg] = {"RMSE": rmse, "MAPE": mape, "MBE": mbe, "R2": r2}
    all_preds[reg] = pd.DataFrame({"Actual": y_true, "Predicted": y_hat})

# -------------------------------------------------------------
# 6. Results Summary
# -------------------------------------------------------------
metrics_df = pd.DataFrame(all_metrics).T
print("\n--- Shape-Scaled Region-wise Metrics (2023) ---")
print(metrics_df.round(3))
metrics_df.to_csv("region_rf_shape_metrics_2023.csv")

# Save predictions
for reg in region_cols:
    all_preds[reg].to_csv(f"{reg.replace(' ', '_')}_rf_shape_predictions_2023.csv")

print("\nSaved all scaled-shape RF results and predictions.")



2.

In [ ]:
# multi_output_xgb_regionwise.py - Predicting 2023 Load using Multi-Output XGBoost per Region
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import skew, kurtosis
from numpy.fft import rfft

# ------------------------------------------------------------- FILE PATHS
TRAIN_XLSX  = "/content/Hourly_Load_Data_2018_2022_LeapFixed.xlsx"
TEST_XLSX   = "/content/India_2023_Hourly_Load_Data.xlsx"
ANNUAL_XLSX = "/content/Regional_Peak___Energy__2018_2023_ &2030.xlsx"
# ------------------------------------------------------------- UTILITIES
def add_timestamp(df):
    year = int(df["Year"].iloc[0])
    idx = pd.date_range(f"{year}-01-01 00:00", periods=len(df), freq="h")
    df.insert(0, "ts", idx)
    return df.set_index("ts")

def fft_feature_array(x, n=5):
    fft_vals = np.abs(rfft(x - np.mean(x)))[: n + 1]
    fft_vals = fft_vals / (np.linalg.norm(fft_vals) + 1e-6)
    return fft_vals[1 : n + 1]

def make_features(df):
    feats = pd.DataFrame(index=df.index)
    for h in [1, 2, 3, 6, 12, 24, 48, 168]:
        feats[f"lag_{h}"] = df["Load"].shift(h)
    feats["peak_yday"]   = df["Load"].rolling(24).max().shift(1)
    feats["energy_yday"] = df["Load"].rolling(24).sum().shift(1)
    feats["hour"]  = df.index.hour
    feats["dow"]   = df.index.dayofweek
    feats["month"] = df.index.month
    feats["wknd"]  = (feats["dow"] >= 5).astype(int)
    feats["year_peak"]   = df["year_peak"]
    feats["year_energy"] = df["year_energy"]
    feats["mean_24"] = df["Load"].rolling(24).mean().shift(1)
    feats["std_24"]  = df["Load"].rolling(24).std().shift(1)
    for i in range(1, 4):
        feats[f"fft_{i}"] = (
            df["Load"].rolling(24)
            .apply(lambda x: fft_feature_array(x)[i - 1] if (~np.isnan(x)).sum() == 24 else np.nan, raw=True)
            .shift(1)
        )
    feats["target"] = df["Load"]
    return feats.dropna()

# ------------------------------------------------------------- LOAD DATA
train_xl  = pd.ExcelFile(TRAIN_XLSX)
train_raw = pd.concat([train_xl.parse(str(y)).assign(Year=y) for y in range(2018, 2023)], ignore_index=True)
train_raw = add_timestamp(train_raw)

test_raw = pd.read_excel(TEST_XLSX, sheet_name="Sheet1")
test_raw["Year"] = 2023
test_raw = add_timestamp(test_raw)

region_cols = ["Northern Region", "North Eastern Region", "Eastern Region", "Southern Region", "Western Region"]

ann = pd.read_excel(ANNUAL_XLSX, engine="openpyxl")
ann.columns = [c.strip().lower() for c in ann.columns]
ann = ann.rename(columns={
    "year": "Year",
    next(c for c in ann.columns if "peak" in c): "year_peak",
    next(c for c in ann.columns if "energy" in c): "year_energy",
})
ann["Year"] = ann["Year"].astype(int)
ann = ann.drop_duplicates("Year").set_index("Year")

for df in (train_raw, test_raw):
    df["year_peak"]   = df["Year"].map(ann["year_peak"])
    df["year_energy"] = df["Year"].map(ann["year_energy"])

# ------------------------------------------------------------- REGION-WISE TRAINING
results = {}

for reg in region_cols:
    print(f"\n=== Training Multi-Output Model: {reg} ===")
    tr_df = train_raw.copy()
    te_df = test_raw.copy()
    tr_df["Load"] = tr_df[reg]
    te_df["Load"] = te_df[reg]

    feat_tr = make_features(tr_df)
    feat_te = make_features(te_df)

    X_tr = feat_tr.drop(columns=["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"])
    Y_tr = feat_tr[["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"]]
    X_te = feat_te.drop(columns=["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"])
    Y_te = feat_te[["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"]]

    model = MultiOutputRegressor(
        XGBRegressor(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            tree_method="hist",
            objective="reg:squarederror",
            random_state=42,
        )
    )

    model.fit(X_tr.astype(np.float32), Y_tr)
    Y_hat = pd.DataFrame(model.predict(X_te.astype(np.float32)), columns=Y_te.columns, index=Y_te.index)

    result = {}
    for col in Y_te.columns:
        result[col] = {
            "RMSE": np.sqrt(mean_squared_error(Y_te[col], Y_hat[col])),
            "R2": r2_score(Y_te[col], Y_hat[col])
        }

    Y_hat.to_csv(f"{reg.replace(' ', '_')}_multioutput_predictions_2023.csv")
    pd.DataFrame(result).T.to_csv(f"{reg.replace(' ', '_')}_multioutput_metrics_2023.csv")
    results[reg] = result

# ------------------------------------------------------------- OVERALL METRICS
metrics_df = pd.concat({k: pd.DataFrame(v).T for k, v in results.items()}, names=["Region", "Target"])
print("\n--- All Region-wise Multi-output Metrics (2023) ---")
print(metrics_df.round(3))
metrics_df.to_csv("regionwise_multioutput_metrics_2023.csv")



=== Training Multi-Output Model: Northern Region ===

=== Training Multi-Output Model: North Eastern Region ===

=== Training Multi-Output Model: Eastern Region ===

=== Training Multi-Output Model: Southern Region ===

=== Training Multi-Output Model: Western Region ===

--- All Region-wise Multi-output Metrics (2023) ---
                                  RMSE     R2
Region               Target                  
Northern Region      target   1212.503  0.988
                     mean_24   535.950  0.997
                     std_24    767.593  0.784
                     fft_1       0.146  0.346
                     fft_2       0.130  0.319
                     fft_3       0.100 -0.139
North Eastern Region target     61.995  0.982
                     mean_24    19.467  0.997
                     std_24     28.516  0.738
                     fft_1       0.071  0.671
                     fft_2       0.103  0.721
                     fft_3       0.064  0.032
Eastern Region       target   

3.

In [ ]:
# forecast_2030_vs_cea_fixed.py
# Forecast 2030 hourly regional load using trained Random Forest models

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from numpy.fft import rfft
from datetime import datetime
import calendar

# --------------------------------------
# Load annual peak + energy
# --------------------------------------
annual_file = "/content/Regional_Peak___Energy__2018_2023_ &2030.xlsx"
ann_df = pd.read_excel(annual_file, engine="openpyxl")
ann_df.columns = [c.strip().lower() for c in ann_df.columns]

# Rename columns
ann_df = ann_df.rename(columns={
    "year": "Year",
    "region": "Region",
    "peak_mw": "year_peak",
    "energy_mu": "year_energy"
})

ann_2030 = ann_df[ann_df["Year"] == 2030].copy()

# --------------------------------------
# Create dummy 2030 input dataframe
# --------------------------------------
region_cols = ann_2030["Region"].tolist()
df_2030_all = {}

for reg in region_cols:
    print(f"Preparing inputs for: {reg}")
    year = 2030
    n_hours = 366 * 24 if calendar.isleap(year) else 365 * 24
    ts = pd.date_range(f"{year}-01-01", periods=n_hours, freq="h")
    df = pd.DataFrame(index=ts)
    df["hour"] = df.index.hour
    df["dow"] = df.index.dayofweek
    df["month"] = df.index.month
    df["wknd"] = (df["dow"] >= 5).astype(int)
    df["Year"] = year
    df["year_peak"] = ann_2030.loc[ann_2030["Region"] == reg, "year_peak"].values[0]
    df["year_energy"] = ann_2030.loc[ann_2030["Region"] == reg, "year_energy"].values[0]
    df_2030_all[reg] = df

# --------------------------------------
# Load & train model using 2018-2022
# --------------------------------------
train_file = "/content/Hourly_Load_Data_2018_2022_LeapFixed.xlsx"
train_xl = pd.ExcelFile(train_file)
train_raw = pd.concat([
    train_xl.parse(str(y)).assign(Year=y) for y in range(2018, 2023)
])

# FFT utilities
def fft_feature_array(x, n=3):
    fft_vals = np.abs(rfft(x - np.mean(x)))[:n+1]
    fft_vals = fft_vals / (np.linalg.norm(fft_vals) + 1e-6)
    return fft_vals[1:n+1]

def make_features(df):
    feats = pd.DataFrame(index=df.index)
    for h in [1, 2, 3, 6, 12, 24, 48, 168]:
        feats[f"lag_{h}"] = df["Load"].shift(h)
    feats["peak_yday"] = df["Load"].rolling(24).max().shift(1)
    feats["energy_yday"] = df["Load"].rolling(24).sum().shift(1)
    feats["hour"] = df.index.hour
    feats["dow"] = df.index.dayofweek
    feats["month"] = df.index.month
    feats["wknd"] = (feats["dow"] >= 5).astype(int)
    feats["year_peak"] = df["year_peak"]
    feats["year_energy"] = df["year_energy"]
    feats["mean_24"] = df["Load"].rolling(24).mean().shift(1)
    feats["std_24"] = df["Load"].rolling(24).std().shift(1)
    for i in range(1, 4):
        feats[f"fft_{i}"] = df["Load"].rolling(24).apply(
            lambda x: fft_feature_array(x)[i-1] if np.isfinite(x).sum() == 24 else np.nan,
            raw=True
        ).shift(1)
    feats["target"] = df["Load"]
    return feats.dropna()

results = []

for reg in region_cols:
    print(f"\n=== Predicting 2030 Load for: {reg} ===")
    train_raw["Load"] = train_raw[reg]
    ann_years = ann_df.drop_duplicates("Year").set_index("Year")
    train_raw["year_peak"] = train_raw["Year"].map(ann_years["year_peak"])
    train_raw["year_energy"] = train_raw["Year"].map(ann_years["year_energy"])
    feat_train = make_features(train_raw)

    X_train = feat_train.drop(columns=["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"])
    Y_train = feat_train[["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"]]

    model = MultiOutputRegressor(RandomForestRegressor(
        n_estimators=200, max_depth=16, min_samples_split=10,
        max_features="sqrt", n_jobs=-1, random_state=42
    ))
    model.fit(X_train, Y_train)

    # Inference 2030
    X_2030 = df_2030_all[reg]
    Y_2030_hat = pd.DataFrame(model.predict(X_2030),
                              columns=Y_train.columns,
                              index=X_2030.index)

    pred_peak = Y_2030_hat["target"].max()
    pred_energy = Y_2030_hat["target"].sum() / 1000  # MW -> MU

    results.append([
        reg,
        pred_peak,
        ann_2030.loc[ann_2030["Region"] == reg, "year_peak"].values[0],
        pred_energy,
        ann_2030.loc[ann_2030["Region"] == reg, "year_energy"].values[0]
    ])

# --------------------------------------
# Show Results
# --------------------------------------
df_comp = pd.DataFrame(results, columns=["Region", "Pred_Peak", "CEA_Peak", "Pred_Energy", "CEA_Energy"])
df_comp.set_index("Region", inplace=True)
df_comp["%Error_Peak"] = 100 * (df_comp["Pred_Peak"] - df_comp["CEA_Peak"]) / df_comp["CEA_Peak"]
df_comp["%Error_Energy"] = 100 * (df_comp["Pred_Energy"] - df_comp["CEA_Energy"]) / df_comp["CEA_Energy"]

print("\n--- 2030 Forecast vs CEA Targets ---")
print(df_comp.round(2))
df_comp.to_csv("forecast_vs_cea_2030.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/Regional_Peak___Energy__2018_2023_ &2030.xlsx'

4.

In [ ]:
# forecast_2030_from_excel.py
# Forecast 2030 hourly regional load using Excel input instead of text input

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from numpy.fft import rfft
import calendar

# ------------------------------------------------------------
# 1. Load Excel input for 2018–2022
# ------------------------------------------------------------
train_file = "/content/Hourly_Load_Data_2018_2022_LeapFixed.xlsx"
train_xl = pd.ExcelFile(train_file)

train_raw = pd.concat([
    train_xl.parse(str(y)).assign(Year=y)
    for y in range(2018, 2023)
], ignore_index=True)

full_index = []
for yr in range(2018, 2023):
    n_hours = 366 * 24 if calendar.isleap(yr) else 365 * 24
    full_index.extend(pd.date_range(f"{yr}-01-01", periods=n_hours, freq="h"))
train_raw.index = full_index[:len(train_raw)]

# ------------------------------------------------------------
# 2. 2030 CEA target values
# ------------------------------------------------------------
regions = ['Northern Region', 'North Eastern Region', 'Eastern Region',
           'Southern Region', 'Western Region']

cea_2030 = {
    "Northern Region":        {"peak": 127553, "energy": 773545},
    "Western Region":         {"peak": 114766, "energy": 763198},
    "Southern Region":        {"peak": 107259, "energy": 596557},
    "Eastern Region":         {"peak": 50420,  "energy": 308103},
    "North Eastern Region":   {"peak": 6519,   "energy": 32373}
}

# ------------------------------------------------------------
# 3. FFT + Feature Builder
# ------------------------------------------------------------
def fft_feature_array(x, n=3):
    fft_vals = np.abs(rfft(x - np.mean(x)))[:n+1]
    fft_vals = fft_vals / (np.linalg.norm(fft_vals) + 1e-6)
    return fft_vals[1:n+1]

def make_features(df, load_col):
    feats = pd.DataFrame(index=df.index)
    for h in [1, 2, 3, 6, 12, 24, 48, 168]:
        feats[f"lag_{h}"] = df[load_col].shift(h)
    feats["peak_yday"] = df[load_col].rolling(24).max().shift(1)
    feats["energy_yday"] = df[load_col].rolling(24).sum().shift(1)
    feats["hour"] = df.index.hour
    feats["dow"] = df.index.dayofweek
    feats["month"] = df.index.month
    feats["wknd"] = (feats["dow"] >= 5).astype(int)
    feats["year_peak"] = df["year_peak"]
    feats["year_energy"] = df["year_energy"]
    feats["mean_24"] = df[load_col].rolling(24).mean().shift(1)
    feats["std_24"] = df[load_col].rolling(24).std().shift(1)
    for i in range(1, 4):
        feats[f"fft_{i}"] = df[load_col].rolling(24).apply(
            lambda x: fft_feature_array(x)[i-1] if np.isfinite(x).sum()==24 else np.nan,
            raw=True
        ).shift(1)
    feats["target"] = df[load_col]
    # Do not dropna here, handle NaNs during prediction
    return feats

# ------------------------------------------------------------
# 4. Train + Forecast (Iterative)
# ------------------------------------------------------------
def forecast_region_2030(reg):
    train_raw["Load"] = train_raw[reg]
    train_raw["year_peak"] = cea_2030[reg]["peak"]
    train_raw["year_energy"] = cea_2030[reg]["energy"] * 1000
    feat_train = make_features(train_raw, "Load").dropna() # Drop NaNs for training

    X_train = feat_train.drop(columns=["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"])
    Y_train = feat_train[["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"]]

    model = MultiOutputRegressor(
        RandomForestRegressor(
            n_estimators=200, max_depth=16, min_samples_split=10,
            max_features="sqrt", n_jobs=-1, random_state=42
        )
    )
    model.fit(X_train, Y_train)

    year = 2030
    n_hours = 366 * 24 if calendar.isleap(year) else 365 * 24
    ts_2030 = pd.date_range(f"{year}-01-01", periods=n_hours, freq="h")
    df_pred = pd.DataFrame(index=ts_2030, columns=["Load"])
    df_pred["hour"] = df_pred.index.hour
    df_pred["dow"] = df_pred.index.dayofweek
    df_pred["month"] = df_pred.index.month
    df_pred["wknd"] = (df_pred["dow"] >= 5).astype(int)
    df_pred["year_peak"] = cea_2030[reg]["peak"]
    df_pred["year_energy"] = cea_2030[reg]["energy"] * 1000

    historical_window_size = 168 # Max lag/rolling window size
    df_seed = train_raw[["Load", "year_peak", "year_energy"]].iloc[-historical_window_size:].copy()
    df_seed["hour"] = df_seed.index.hour
    df_seed["dow"] = df_seed.index.dayofweek
    df_seed["month"] = df_seed.index.month
    df_seed["wknd"] = (df_seed["dow"] >= 5).astype(int)

    df_full = pd.concat([df_seed, df_pred])
    df_full["Load"] = df_full["Load"].astype(float)

    for t in ts_2030:
        # Use a window that includes the necessary historical data for feature calculation
        window = df_full.loc[:t].iloc[-historical_window_size-1:]
        feats = make_features(window, "Load")

        # Check if features were successfully created for the current timestamp
        if t in feats.index:
            X_t = feats.loc[[t]].drop(columns=["target", "mean_24", "std_24", "fft_1", "fft_2", "fft_3"])
            y_hat = model.predict(X_t)[0]
            df_full.at[t, "Load"] = y_hat[0] # Update df_full with predicted load
        else:
            # Handle cases where features cannot be calculated (e.g., early hours of forecast)
            # For simplicity, we can carry over the last known load or use a default
            # Here, we'll just note it. More sophisticated imputation might be needed.
            print(f"Could not calculate features for {t}. Using previous hour's load.")
            df_full.at[t, "Load"] = df_full.loc[df_full.index < t, "Load"].iloc[-1]


    return df_full.loc[ts_2030, "Load"] # Return only the 2030 predictions


# ------------------------------------------------------------
# 5. Forecast all regions + compare
# ------------------------------------------------------------
results = []
for reg in regions:
    print(f"\n=== Forecasting 2030 for {reg} ===")
    y_hat = forecast_region_2030(reg)
    pred_peak = y_hat.max()
    pred_energy = y_hat.sum() / 1000
    results.append([
        reg,
        pred_peak,
        cea_2030[reg]["peak"],
        pred_energy,
        cea_2030[reg]["energy"]
    ])

# ------------------------------------------------------------
# 6. Results Table
# ------------------------------------------------------------
df_comp = pd.DataFrame(results, columns=[
    "Region", "Pred_Peak", "CEA_Peak", "Pred_Energy", "CEA_Energy"])
df_comp.set_index("Region", inplace=True)
df_comp["%Error_Peak"] = 100 * (df_comp["Pred_Peak"] - df_comp["CEA_Peak"]) / df_comp["CEA_Peak"]
df_comp["%Error_Energy"] = 100 * (df_comp["Pred_Energy"] - df_comp["CEA_Energy"]) / df_comp["CEA_Energy"]

print("\n--- 2030 Forecast vs CEA Targets ---")
print(df_comp.round(2))
df_comp.to_csv("forecast_vs_cea_2030_from_excel_iterative.csv")


=== Forecasting 2030 for Northern Region ===


/tmp/ipython-input-1767314075.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_full = pd.concat([df_seed, df_pred])



=== Forecasting 2030 for North Eastern Region ===


/tmp/ipython-input-1767314075.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_full = pd.concat([df_seed, df_pred])



=== Forecasting 2030 for Eastern Region ===


/tmp/ipython-input-1767314075.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_full = pd.concat([df_seed, df_pred])



=== Forecasting 2030 for Southern Region ===


/tmp/ipython-input-1767314075.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_full = pd.concat([df_seed, df_pred])
